In [8]:
'''
8x8 = 64
9x9 = 81
10x10 = 100
10x8x2 = 160
10x9x2 = 180
8x8x4 = 256
8x8x4 = 256
8x9x4 = 288
9x9x4 = 324
8x9x2 = 144
8x9x4 = 288
 2141 first moves
'''

['abc', 'd', 'e']
True


In [42]:
# Assuming piece_layouts is a dictionary mapping piece_id to a 4x4 numpy array
# pyqt
# https://github.com/Rafael1s/Deep-Reinforcement-Learning-Algorithms/blob/master/LunarLander-v2-DQN/WatchAgent-DQN.ipynb
import numpy as np

piece_layouts = {
    'cathedral': np.array([[0, 1, 0],
                            [1, 1, 1],
                            [0, 1, 0],
                            [0, 1, 0]]),

    'tavern': np.array([[1]]),

    'stable': np.array([[1, 1]]),

    'inn': np.array([[1, 0],
                 [1, 1]]),

    'bridge': np.array([[1, 1, 1]]),

    'square': np.array([[1, 1],
                        [1, 1,]]),

    'manor': np.array([[1, 1, 1],
                        [0, 1, 0]]),

    'abbey_w': np.array([[0, 1, 1],
                        [1, 1, 0]]),

    'abbey_b': np.array([[1, 1, 0],
                        [0, 1, 1]]),

    'academy_w': np.array([[1, 1, 0],
                            [0, 1, 1],
                            [0, 1, 0]]),

    'academy_b': np.array([[0, 1, 1],
                 [1, 1, 0],
                 [0, 1, 0]]),                 
    
    'infirmary': np.array([[0, 1, 0],
                            [1, 1, 1],
                            [0, 1, 0]]),
    'castle': np.array([[1, 1, 1],
                        [1, 0, 1]]),
    'tower': np.array([[1, 0, 0],
                        [1, 1, 0],
                        [0, 1, 1]])
}

import numpy as np


def rotate(piece, n):
    if n == 0:
        return piece
    return rotate(piece.copy().T[::-1], n - 1)

def all_rotations_generator(piece):
    yield piece
    for n in range(1, 4):
        piece = piece.copy().T[::-1]  # Rotate the piece
        yield piece

In [7]:
import random
import typing
from termcolor import colored

class State:
    def __init__(self, board, ap, bench_w = None, bench_b = None):
        '''
        board key:
        0: empty
        1: occupied w
        2: occupied b
        3: occupied cathedral
        4: owned w
        5: owned b
        '''
        self.board, self.ap, self.bench_w, self.bench_b = board, ap, bench_w, bench_b

    def __str__(self):
        color_map = {
            0: 'black',
            1: 'red',
            2: 'blue',
            3: 'green',
            4: 'light_red',
            5: 'light_blue'
        }

        colored_board = [[colored(str(cell), color_map[cell]) for cell in row] for row in self.board]
        return "\n".join(" ".join(cell for cell in row) for row in colored_board)



type coordinate = tuple[str, int]
type board = np.ndarray[np.integer, (10, 10)]
type piece = np.ndarray[np.integer]
type bench = list[np.ndarray]
type bench = list[piece]
type state = tuple[board, int, bench, bench]

class ActionSpace:

    @staticmethod
    def __placement_mask(board: board, ap: int) -> board:
        if ap == 0:
            # white
            return np.isin(board, [1, 2, 3, 5]).astype(int)
        else:
            return np.isin(board, [1, 2, 3, 4]).astype(int)
    
    @staticmethod
    def can_place_piece(state, piece, row, col) -> bool:
        mask = ActionSpace.__placement_mask(state.board, state.ap)
        
        if row + piece.shape[0] > state.board.shape[0] or col + piece.shape[1] > state.board.shape[1]:
            return False
        if np.any(mask[row:row+piece.shape[0], col:col+piece.shape[1]] & piece):
            return False
        return True

    # list of the updated squares upon piece placement
    @staticmethod
    def coords_changed_by_placement(piece, row, col) -> list[coordinate]:
        indices = np.where(piece == 1)
        row_indices = indices[0] + row
        col_indices = indices[1] + col
        coordinates = list(zip(row_indices, col_indices))
        return coordinates

    def calculate_ownership(state: state, piece: piece, row, col) -> board:
        updated = ActionSpace.coords_changed_by_placement(piece, row, col)
        
        

    @staticmethod
    def place_piece(state: state, piece: piece, row, col) -> board:
        mask = piece == 1
        state.board[row:row+piece.shape[0], col:col+piece.shape[1]] = np.where(piece == 1, 1 if state.ap==0 else 2, state.board[row:row+piece.shape[0], col:col+piece.shape[1]])
        
        # remove placed piece from bench
        for i, bench_piece in enumerate(ActionSpace.activeBench(state)):
            for rotated_piece in all_rotations_generator(piece):
                if np.array_equal(bench_piece, rotated_piece):
                    del ActionSpace.activeBench(state)[i]
                    break
        state.ap = (state.ap + 1) % 2

    @staticmethod
    def get_n_placements(state, piece: piece, n) -> list[coordinate]:
        # warning: will return multiple instances of placing same piece
        piece_rotations = all_rotations_generator(piece)
        moves = [(p, x, y) for p in piece_rotations for x in range(state.board.shape[0]-p.shape[0]+1) for y in range(state.board.shape[0]-p.shape[0] + 1)]
        random.shuffle(moves)
        retVal = []

        for move in moves:
            if ActionSpace.can_place_piece(state, *move):
                retVal.append(move)
                if len(retVal) >= n:
                    return retVal
        return None
    
    @staticmethod
    def activeBench(state: State) -> bench:
        if state.ap == 0:
            return state.bench_w
        return state.bench_b

In [80]:
import random
w_bench = ['tavern', 'tavern', 'stable', 'stable', 'inn', 'inn', 'bridge', 'square', 'manor', 'abbey_w', 'academy_w', 'infirmary', 'castle', 'tower']
b_bench = ['tavern', 'tavern', 'stable', 'stable', 'inn', 'inn', 'bridge', 'square', 'manor', 'abbey_b', 'academy_b', 'infirmary', 'castle', 'tower']
w_bench_small = ['stable', 'abbey_w',]
b_bench_small = ['stable', 'abbey_b']
s = State(np.zeros((10, 10), dtype=int), ap = 0, bench_w = [piece_layouts[key] for key in w_bench_small], bench_b = [piece_layouts[key] for key in b_bench_small])

for i in range(4):
    piece = ActionSpace.activeBench(s)[0]
    #piece = random.choice(a.activeBench(s))
    ActionSpace.place_piece(s, *ActionSpace.get_n_placements(s, piece, 1)[0])
print(ActionSpace.activeBench(s))
print(s)

[]
0 0 0 0 0 0 0 1 1 0
0 0 0 0 0 0 1 1 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 2 2 0 0 0
0 0 0 0 0 0 0 0 0 0
0 2 0 0 0 0 0 0 0 0
2 2 0 0 0 0 0 0 0 0
2 0 0 0 0 0 0 0 0 0
0 0 0 1 1 0 0 0 0 0


In [182]:
p = np.array([[1, 1]])
bench = [np.array([[1]]), np.array([[1, 1]])]

print(p in bench)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [46]:
import random
import torch
import torch.nn as nn
import torch.optim as optim
use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
device = torch.device("cuda" if use_cuda else "cpu")

class QNetwork(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim) -> None:
        """DQN Network

        Args:
            input_dim (int): `state` dimension.
                `state` is 2-D tensor of shape (n, input_dim)
            output_dim (int): Number of actions.
                Q_value is 2-D tensor of shape (n, output_dim)
            hidden_dim (int): Hidden dimension in fc layer
        """
    
        super(QNetwork, self).__init__()

        self.layer1 = torch.nn.Sequential(
            torch.nn.Linear(input_dim, hidden_dim),
            torch.nn.PReLU()
        )

        self.layer2 = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim, hidden_dim),
            torch.nn.PReLU()
        )

        self.layer3 = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim, hidden_dim),
            torch.nn.PReLU()
        )

        self.final = torch.nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Returns a Q_value

        Args:
            x (torch.Tensor): `State` 2-D tensor of shape (n, input_dim)

        Returns:
            torch.Tensor: Q_value, 2-D tensor of shape (n, output_dim)            
        """
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.final(x)

        return x




BATCH_SIZE = 64  
LEARNING_RATE = 0.001

class Agent(object):

    def __init__(self, n_states, n_actions, hidden_dim):
        """Agent class that choose action and train

        Args:
            n_states (int): input dimension
            n_actions (int): output dimension
            hidden_dim (int): hidden dimension
        """
        
        # calculating input dimension: board size + active player + player benches +
        input_dim = (10*10) + 1 + (2*14)

        self.q_local = QNetwork(input_dim, n_actions, hidden_dim=16).to(device)
        self.q_target = QNetwork(n_states, n_actions, hidden_dim=16).to(device)
        
        self.mse_loss = torch.nn.MSELoss()
        self.optim = optim.Adam(self.q_local.parameters(), lr=LEARNING_RATE)
        
        self.n_states = input_dim
        self.n_actions = n_actions
        

    def get_action(self, state, eps, check_eps=True):
        """Returns an action

        Args:
            state : 2-D tensor of shape (n, input_dim)
            eps (float): eps-greedy for exploration

        Returns: int: action index
        """
        global steps_done
        sample = random.random()

        if check_eps==False or sample > eps:
            with torch.no_grad():
                return self.q_local(Variable(state).type(FloatTensor)).data.max(1)[1].view(1, 1)
        else:
           ## return LongTensor([[random.randrange(2)]])
           return torch.tensor([[random.randrange(self.n_actions)]], device=device) 


    def learn(self, experiences, gamma):
        """Prepare minibatch and train them

        Args:
        experiences (List[Transition]): batch of `Transition`
        gamma (float): Discount rate of Q_target
        """
        
        if len(self.replay_memory.memory) < BATCH_SIZE:
            return;
            
        transitions = self.replay_memory.sample(BATCH_SIZE)
        
        batch = Transition(*zip(*transitions))
                        
        states = torch.cat(batch.state)
        actions = torch.cat(batch.action)
        rewards = torch.cat(batch.reward)
        next_states = torch.cat(batch.next_state)
        dones = torch.cat(batch.done)
        
            
        # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
        # columns of actions taken. These are the actions which would've been taken
        # for each batch state according to newtork q_local (current estimate)
        Q_expected = self.q_local(states).gather(1, actions)     

        Q_targets_next = self.q_target(next_states).detach().max(1)[0] 

        # Compute the expected Q values
        Q_targets = rewards + (gamma * Q_targets_next * (1-dones))
        
        self.q_local.train(mode=True)        
        self.optim.zero_grad()
        loss = self.mse_loss(Q_expected, Q_targets.unsqueeze(1))
        # backpropagation of loss to NN        
        loss.backward()
        self.optim.step()
               
        
    def soft_update(self, local_model, target_model, tau):
        """ tau (float): interpolation parameter"""
        
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)     
            
    def hard_update(self, local, target):
        for target_param, param in zip(target.parameters(), local.parameters()):
            target_param.data.copy_(param.data)            
            

In [68]:
import numpy as np

jordy_odds = 0.64
opp_odds = 1 - jordy_odds

jordywins = 0
oppwins = 0
for i in range(1000000):
    curr_jordywins = 0
    curr_oppwins = 0
    while True:
        winner = np.random.choice(['j', 'o'], p = [jordy_odds, opp_odds])
        if winner == 'j':
            curr_jordywins += 1
        else:
            curr_oppwins += 1
        if curr_jordywins == 3 or curr_oppwins == 3:
            break
    if curr_jordywins == 3:
        jordywins += 1
    else:
        curr_oppwins += 1
print(jordywins/1000000)

0.748913


In [70]:
gamesPlayed = 0
n = 1000000
for i in range(n):
    currGamesPlayed = 0
    curr_jordywins = 0
    curr_oppwins = 0
    while True:
        currGamesPlayed += 1
        winner = np.random.choice(['j', 'o'])
        if winner == 'j':
            curr_jordywins += 1
        else:
            curr_oppwins += 1
        if curr_jordywins == 2 or curr_oppwins == 2:
            break
    gamesPlayed += currGamesPlayed

print(gamesPlayed/n)

2.500124
